In [1]:
import pandas as pd
import numpy as np
import gc
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import KFold, StratifiedKFold, cross_val_score
from datetime import datetime
#from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler,LabelEncoder, OneHotEncoder, minmax_scale, scale, LabelBinarizer
from sklearn import tree
from sklearn import linear_model
from sklearn import svm
from sklearn import neighbors
from sklearn import ensemble
from sklearn.feature_selection import SelectFromModel, VarianceThreshold,RFE, f_regression
#from minepy import MINE
from mlxtend.regressor import StackingRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import gc
color = sns.color_palette()
from mlxtend.regressor import StackingRegressor, StackingCVRegressor
import tensorflow as tf

E:\python-ana\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
sample=pd.read_csv('D://f_sample_20180204.csv')
train=pd.read_csv('D://f_train_20180204.csv',encoding='gbk')
test=pd.read_csv('D://f_test_a_20180204.csv',encoding='gbk')

In [3]:
train.columns.values

array(['id', 'SNP1', 'SNP2', 'SNP3', 'SNP4', 'SNP5', 'SNP6', 'SNP7',
       'SNP8', 'SNP9', 'SNP10', 'SNP11', 'SNP12', 'SNP13', 'SNP14',
       'SNP15', 'SNP16', 'SNP17', 'SNP18', 'SNP19', 'SNP20', 'SNP21',
       'SNP22', 'SNP23', 'RBP4', '年龄', '孕次', '产次', '身高', '孕前体重', 'BMI分类',
       '孕前BMI', '收缩压', '舒张压', '分娩时', '糖筛孕周', 'VAR00007', 'wbc', 'ALT',
       'AST', 'Cr', 'BUN', 'CHO', 'TG', 'HDLC', 'LDLC', 'ApoA1', 'ApoB',
       'Lpa', 'hsCRP', 'SNP24', 'SNP25', 'SNP26', 'SNP27', 'SNP28',
       'SNP29', 'SNP30', 'SNP31', 'SNP32', 'SNP33', 'SNP34', 'SNP35',
       'SNP36', 'SNP37', 'SNP38', 'DM家族史', 'SNP39', 'SNP40', 'SNP41',
       'SNP42', 'SNP43', 'SNP44', 'SNP45', 'SNP46', 'SNP47', 'SNP48',
       'SNP49', 'SNP50', 'SNP51', 'SNP52', 'SNP53', 'SNP54', 'SNP55',
       'ACEID', 'label'], dtype=object)

In [4]:
def my_dropna(df):
    des=df.describe().ix[0]
    index=0
    for col in df.columns.values:
        if des[index]/len(df[col]) <= 0.81:
            df.drop(col,axis=1,inplace=True)
        index=index+1

In [5]:
def my_fillna(df):
    df.fillna(df.mean(),inplace=True)

In [6]:
my_dropna(train)
my_fillna(train)

E:\python-ana\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [7]:
label_train=train['label']
train.drop('label',axis=1,inplace=True)
train.drop('id',axis=1,inplace=True)

In [8]:
train

,SNP1,SNP2,SNP3,SNP4,SNP5,SNP6,SNP7,SNP8,SNP9,SNP10,...,SNP44,SNP45,SNP46,SNP47,SNP48,SNP49,SNP50,SNP51,SNP52,SNP53
0,2,1,2.796414,2.0,3.000000,1.0,3.0,3.0,2.0,2.0,...,2.000000,2.000000,1.000000,2.000000,2.0,2.000000,1.000000,2.000000,1.0,1.000000
1,3,2,3.000000,2.0,2.000000,2.0,3.0,3.0,2.0,2.0,...,1.000000,1.000000,2.000000,2.000000,1.0,3.000000,1.000000,1.000000,2.0,1.000000
2,1,1,3.000000,1.0,3.000000,2.0,3.0,3.0,2.0,2.0,...,1.000000,3.000000,2.000000,2.000000,3.0,3.000000,2.000000,2.000000,1.0,1.000000
3,1,3,3.000000,1.0,2.098396,1.0,3.0,1.0,1.0,1.0,...,2.000000,1.000000,2.000000,2.000000,1.0,3.000000,1.000000,2.000000,1.0,1.000000
4,1,2,3.000000,2.0,2.000000,1.0,2.0,2.0,1.0,2.0,...,1.000000,2.000000,1.000000,1.000000,3.0,1.000000,2.000000,2.000000,1.0,1.000000
5,2,2,3.000000,2.0,1.000000,2.0,3.0,2.0,1.0,2.0,...,1.000000,2.000000,1.000000,2.000000,2.0,2.000000,1.000000,1.000000,2.0,1.000000
6,2,2,3.000000,1.0,3.000000,1.0,3.0,3.0,2.0,2.0,...,1.000000,2.000000,1.000000,3.000000,3.0,2.000000,1.000000,2.000000,1.0,1.000000
7,3,2,3.000000,2.0,2.000000,2.0,3.0,2.0,1.0,2.0,...,1.000000,2.000000,3.000000,1.905561,1.0,3.000000,1.000000,2.000000,1.0,1.000000
8,2,2,3.000000,1.0,1.000000,1.0,3.0,3.0,2.0,2.0,...,1.000000,1.000000,1.000000,1.000000,1.0,2.000000,1.000000,1.000000,2.0,1.000000
9,1,2,3.000000,2.0,2.000000,2.0,2.0,3.0,2.0,2.0,...,2.000000,3.000000,3.000000,2.000000,3.0,3.000000,1.000000,1.000000,2.0,1.000000


In [9]:
np.array(train.ix[0]).reshape([8,8])

E:\python-ana\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


array([[  2.        ,   1.        ,   2.7964135 ,   2.        ,
          3.        ,   1.        ,   3.        ,   3.        ],
       [  2.        ,   2.        ,   2.        ,   1.        ,
          1.        ,   1.        ,   2.        ,   3.        ],
       [  2.        ,   3.        ,   3.        ,   1.        ,
         26.        ,   1.57484647,   7.56      ,  55.        ],
       [ 52.22      ,   2.21      ,   4.97      ,   2.4       ,
          1.71      ,   2.82      ,  21.7       ,   9.7       ],
       [ 12.        ,   2.08      ,   1.15862808,   1.        ,
          1.        ,   2.        ,   1.        ,   2.        ],
       [  1.        ,   2.        ,   2.        ,   1.        ,
          3.        ,   1.        ,   2.        ,   1.        ],
       [  3.        ,   2.        ,   1.        ,   2.        ,
          2.        ,   3.        ,   2.        ,   2.        ],
       [  1.        ,   2.        ,   2.        ,   2.        ,
          1.        ,   2.       